In [1]:
# prerequisite: 
# run 
#      ./printEnergyCorrections -s ../../setup/setup.json -t ../../setup/target.json -d U4,U5 -E {insert peaks one wishes to fit after} -i {implantation depth of given isotope} > {filename}.dat
# with data output to
#      "correctedEnergiesU4U5.dat"

In [2]:
import numpy as np
import uproot
import matplotlib.pyplot as plt

In [3]:
#load in generated file from above
Es = np.loadtxt('/home/haue/repositories/Masters/calibration/dssd_calibration/U4_sipeaks_corrected.dat').flatten()
print(len(Es)/4)

56.0


In [4]:
from scipy.optimize import curve_fit
def lin(x, a, b):
    return a*x + b

In [5]:
"""
([(385.72237105453763, 0.5786771607068808),
  (1843.1815428519299, 3.8391338032060935),
  (2217.4549852704345, 1.9202145004587217),
  (4651.192036484095, 4.798722937645029),
  (5402.617947801247, 1.9202145004587354)],
 [(904.0191131421628, 1.920220495723927),
  (2076.7425660534773, 3.8391368018535106),
  (3337.756164355824, 4.798725336660437),
  (4089.1820756729758, 1.920220495723936)],
 (1730.8841553184843, 3.8391483166378255),
"""
nums =  ["4"]
sides = ["F", "B"]
Es0 = np.array([385.72,904.02, 1843.18,2076.74,2217.45,3337.75,4089.18,5402.61])
Es0_errs = np.array([0.58,1.92, 3.84, 3.84, 1.92, 4.79, 1.92, 1.92])

j = 0
for num in nums:
        for side in sides:
            for strip in range(2, 15+1, 1):
                
                print([Es[8*j+i] for i in range(len(Es0))])
                j += 1

[375.797, 897.771, 1839.29, 2073.12, 2213.97, 3335.14, 4086.97, 5400.78]
[375.988, 897.894, 1839.37, 2073.19, 2214.04, 3335.19, 4087.01, 5400.82]
[376.141, 897.993, 1839.43, 2073.25, 2214.1, 3335.24, 4087.05, 5400.85]
[376.252, 898.066, 1839.48, 2073.3, 2214.14, 3335.27, 4087.07, 5400.87]
[376.315, 898.11, 1839.51, 2073.32, 2214.17, 3335.29, 4087.09, 5400.88]
[376.326, 898.121, 1839.51, 2073.33, 2214.18, 3335.29, 4087.1, 5400.89]
[376.279, 898.096, 1839.5, 2073.32, 2214.16, 3335.28, 4087.09, 5400.88]
[376.167, 898.032, 1839.46, 2073.28, 2214.13, 3335.26, 4087.07, 5400.86]
[375.983, 897.924, 1839.4, 2073.22, 2214.07, 3335.22, 4087.03, 5400.84]
[375.719, 897.767, 1839.3, 2073.14, 2213.99, 3335.16, 4086.98, 5400.79]
[375.362, 897.554, 1839.17, 2073.02, 2213.87, 3335.07, 4086.91, 5400.73]
[374.897, 897.276, 1839.01, 2072.86, 2213.72, 3334.96, 4086.82, 5400.66]
[374.303, 896.921, 1838.79, 2072.66, 2213.53, 3334.82, 4086.7, 5400.56]
[373.545, 896.47, 1838.52, 2072.41, 2213.29, 3334.64, 4086.

In [7]:
dead_strips = ["U5F2"]

nums =  ["4"]
sides = ["F", "B"]
j = 0
res0 = res = np.array([])
with uproot.open("/home/haue/repositories/Masters/data/cal/pha.root") as file:
    for num in nums:
        for side in sides:
            if side == "F":
                print("# Front")
            else:
                print("# Back")
            for strip in range(1, 16+1, 1):
                if strip == 1 or strip == 16:
                    print(0, 0)
                    continue
                base = "U" + num + side + str(strip)
                #skip dead strips
                if base in dead_strips:
                    print(0, 0)
                    j += 1
                    continue
                # data
                y1, x1 = file[base].to_numpy()
                x0 = np.min(x1) ## starting bin, in this case just 90
                x1 = x1[:-1] + 0.5
                # tspectrum
                y2, x2 = file[base + "A"].to_numpy()
                x2 = x2[:-1] + 0.5 + x0
                # peak positions, in found order
                x3, y3 = file[base + "P"].values()
                x3 = x3 + x0
                # fitted peak positions and their errors
                fx, fx_err = file[base + "F"].values()

                
                # peak positions, sorted xmin to xmax
                x3_sortedx = x3[np.argsort(x3)] # sorted by low to high x
                y3_sortedx = y3[np.argsort(x3)]
                fx = fx[np.argsort(fx)]
                fx_err = fx_err[np.argsort(fx)]
                
                if len(x3_sortedx)!=len(fx):
                    raise ValueError(f"Length mismatch: len(x3_sortedx)={len(x3_sortedx)} is not equal to len(fx)={len(fx)}, somehow the fitting procedure messed up..")



                # proton peaks pV, pVI, ..., pVIII
                while len(x3_sortedx)>len(Es0):
                    min_y_index = np.argmin(y3_sortedx)
                    x3_sortedx = np.delete(x3_sortedx,min_y_index)
                    y3_sortedx = np.delete(y3_sortedx,min_y_index)
                    fx = np.delete(fx,min_y_index)
                    fx_err = np.delete(fx_err,min_y_index)
                    
                
                ### the gaussian fitting routine is very simple and therefore one should expect errors
                ### This would explain why x3_sortedx is the better parameter to use for now
                #chs = fx
                chs = x3_sortedx
                popt0, pcov0 = curve_fit(lin, chs, Es0, sigma=Es0_errs, p0=[2, 0])
                Ecs = np.array([Es[8*j+i] for i in range(len(x3_sortedx))])
                popt, pcov = curve_fit(lin, chs, Ecs, sigma=Es0_errs, p0=[2, 0])
                
                res0 = np.append(res0, Es0 - lin(chs, *popt0)) ## not energy corrected peak energies
                res  = np.append(res, Ecs - lin(chs, *popt)) ## energy corrected peak energies ! the most important one!
                
                ######################################################################################################################
                ## plotting routine for quality control
                """
                fig, axes = plt.subplots(2, 1,gridspec_kw={'height_ratios': [4, 1], 'hspace': 0}, sharex=True)
                ax1, ax2 = axes
                #plt.sca(axes[0]) ## sets current axes
                ax1.errorbar(chs, Ecs, yerr=Es0_errs, capsize=2, marker='o',markersize=3.5, ls='', color='r', label="Data points")
                ax1.plot([50, 2000], lin(np.array([50, 2000]), *popt), label=f"fit vals: {np.round(popt[1],6)} {np.round(popt[0],6)}", color='k', ls='--')
                ax1.legend()
                ax1.set_ylabel("Energy(keV)")
                ax1.set_title(f"Linear fit to {base}")
                #plt.sca(axes[1])
                ax2.scatter(chs, Ecs - lin(chs, *popt), color='k', s=8)
                ax2.hlines(0,50,2000, ls='--', color='k', lw=0.7)
                ax2.set_yticks(np.arange(-10,10, 2.5), minor=True)
                ax2.set_ylim(-np.max(np.abs(Ecs - lin(chs, *popt)))-2, np.max(np.abs(Ecs - lin(chs, *popt)))+2)
                
                ax2.set_ylabel("Residuals (keV)")
                ax2.set_xlabel("Channel")
                
                plt.show()
                
                
                """
                #### output
                print(popt[1], popt[0])
                j += 1


# Front
0 0
-33.91776207253419 3.504569350779493
-40.36762178563546 3.6169498296313547
-34.57533437908256 3.5079809997994227
-35.86534009074773 3.7139353520927125
-27.710048460223916 3.538902949484369
-40.62954450129258 3.6581893994702144
-32.03876078890981 3.6159117713524163
-27.415296187261458 3.6019759058938026
-19.96853818314437 3.700160398538857
-30.18265753338627 3.6517425862108372
-27.302474173754494 3.7605231866243334
-30.15028731435201 3.604955493313214
-29.74890384571252 3.538906266888517
-24.74358611243713 3.720807375124213
0 0
# Back
0 0
-38.888019128983146 3.5265869437403765
-32.20318617223738 3.626779866419441
-31.95419316857937 3.567083796022153
-30.52836152669454 3.6525228184371947
-37.35206111004597 3.5257125524740354
-36.364656859370726 3.7415071945506444
-28.99058441663953 3.780781684815327
-31.172632951793805 3.6627182828818476
-38.58235580046643 3.5715752152490277
-39.56594270631414 3.6636780129445565
-36.25273293535571 3.6719218421366593
-32.9572516659095 3.608371

In [ ]:
np.savetxt("res_proton_uncorrected.dat", res0)
np.savetxt("res_proton_corrected.dat", res)